<a href="https://colab.research.google.com/github/Victorlouisdg/simulators/blob/main/cloth_Baraff_Witkin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install meshzoo

In [ ]:
import numpy as np
import meshzoo
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
matplotlib.rc('animation', html='jshtml')

In [ ]:
points, cells = meshzoo.rectangle_tri(
    (0.0, 0.0),
    (1.0, 1.0),
    n=11,  # or (11, 11)
    variant="zigzag",  # or "up", "down", "center"
)

In [ ]:
x, z = np.transpose(points)
y = np.zeros_like(x)

In [ ]:
fig = plt.figure(figsize=(8, 6), dpi=100)
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(x, y, z, triangles=cells)
ax.scatter(x, y, z, c='red', s=10)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z');